In [0]:
import requests

date="2025-05-20"
base= "BRL"


url = (f"https://api.apilayer.com/exchangerates_data/{date}?base={base}")

payload = {}
headers= {
  "apikey": "JxSTDnwHcbXtWh67iqZjlcLBQUhlC7Wc"
}

response = requests.request("GET", url, headers=headers, data = payload)

if status_code != 200:
  raise Exception(f"Request failed with status code {status_code}")
status_code = response.status_code
result = response.text

In [0]:
print(result)

In [0]:
import requests

def extraindo_dados(date, base="BRL"):

  url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

  headers = {
    "apikey": "JxSTDnwHcbXtWh67iqZjlcLBQUhlC7Wc"
    }

  parametros = {
    "base":base
    # ,"symbols": "USD,GBP,EUR" # "USD" (Dólar), "GBP" (Libra) e "EUR" (Euro)
    }
 
  response = requests.request("GET", url, headers=headers, params = parametros)

  if response.status_code != 200:
    raise Exception("Não consegui extrair dados!!!")

  return response.json()

In [0]:
data_extracao= "2025-07-25" 
extraindo_dados(data_extracao, base="BRL")

In [0]:
#armazenando o dia da extração para criar as pastas
ano, mes, dia = extraindo_dados(data_extracao)['date'].split('-')
print(ano, mes, dia)

In [0]:
#verificando as pastas do databricks
display(dbutils.fs.ls("dbfs:/databricks-results/"))

In [0]:
#dbfs/Repos/hugo.har@hotmail.com/airflowDatabricsAlura/dados
#/Workspace/Users/hugo.har@hotmail.com/airflowDatabricsAlura
!ls ../

In [0]:
#criando pasta de dados dentro da pasta do github
!mkdir ../dados

In [0]:
caminho_completo = f"dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}"
print(caminho_completo)

In [0]:
#func de extração e tratamento dos dados da moeda criando uma lista de tuplas
def dados_para_dataframe (dado_json): 
        dados_tupla = [(moeda, float (taxa)) for moeda, taxa in dado_json["rates"].items()]
        return dados_tupla

In [0]:
response= dados_para_dataframe(extraindo_dados(data_extracao))

In [0]:
df_conversoes= spark.createDataFrame(response,schema=["moeda", "taxa"])
df_conversoes.show(5)

In [0]:
from pyspark.sql.functions import lit
#df_conversoes.withColumn("data", lit(f"{ano}-{mes}-{dia}")).show(5)
df_conversoes= df_conversoes.withColumn("data", lit(f"{ano}-{mes}-{dia}"))

In [0]:
df_conversoes.write.format("parquet")\
    .mode("overwrite")\
    .save(caminho_completo)

In [0]:
#validando se foi salvo
display(dbutils.fs.ls(f"{caminho_completo}"))